In [185]:
import numpy as np

In [186]:
with open("input12.txt", "r") as file:
    data = file.read().splitlines()
    data = [list(item) for item in data]
    data = np.asmatrix(data)

matrix([['L', 'L', 'L', ..., 'W', 'W', 'W'],
        ['L', 'L', 'L', ..., 'W', 'W', 'W'],
        ['L', 'L', 'L', ..., 'W', 'W', 'W'],
        ...,
        ['G', 'G', 'G', ..., 'S', 'S', 'G'],
        ['G', 'G', 'G', ..., 'S', 'S', 'G'],
        ['G', 'E', 'G', ..., 'S', 'S', 'G']], dtype='<U1')

In [187]:
direction = {0: (0,1),
             1: (1,0),
             2: (0,-1),
             3: (-1,0)}

In [188]:
def flatten_nested_list(nested_list):
    result = []
    for element in nested_list:
        if isinstance(element, list):
            result.extend(flatten_nested_list(element))
        else:
            result.append(element)
    return result

In [189]:
data_new = data.copy()
def find_region(i, j):
    res = [(i,j)]
    n, m = data.shape
    var = data_new[i,j]
    data_new[i,j] = " "
    for k in range(4):
        coords =  []
        x, y = direction[k]
        
        if (n > i+x >= 0 and
            m > j+y >= 0 and
            data_new[i+x, j+y] == var):
            coords.append(find_region(i+x, j+y))
        res.append(coords)
    return(res)        


In [190]:
regions = []
all_region_coords = []
n, m = data.shape

for i in range(n):
    for j in range(m):
        if (i,j) in all_region_coords:
            pass
        else:
            region = find_region(i, j)
            flatten_region = flatten_nested_list(region)
            regions.append(flatten_region)
            all_region_coords.append(flatten_region)
            all_region_coords = flatten_nested_list(all_region_coords)
for i in range(len(regions)):
    regions[i] = list(set(regions[i]))

In [191]:
def find_sides(region, data):
    n, m = data.shape
    sides = []
    for coord in region:
        for i in range(4):
            x, y = direction[i]
            if (n > coord[0]+x >= 0 and
                m > coord[1]+y >= 0 and
                data[coord[0]+x, coord[1]+y] != data[coord[0],coord[1]]):
                sides.append((coord[0]+x, coord[1]+y, coord[0], coord[1]))
            elif (not(n > coord[0]+x >= 0) or
                  not(m > coord[1]+y >= 0)):
                sides.append((coord[0]+x, coord[1]+y, coord[0], coord[1]))
    sides = flatten_nested_list(sides)
    #sides = list(set(sides))
    return sides

In [195]:
def find_connected_sides(reg_sides, i, j, ori_x, ori_y, hv):
    res = [(i, j, ori_x, ori_y)]
    for k in range(len(reg_sides)):
        if reg_sides[k][:2] == (i,j):
            pass
        if (hv == "h" and
            reg_sides[k][0] == i and
            reg_sides[k][1] == j+1 and 
            reg_sides[k][2] == ori_x and 
            reg_sides[k][3] == ori_y+1):
            res.append(find_connected_sides(reg_sides, 
                                            reg_sides[k][0], 
                                            reg_sides[k][1], 
                                            reg_sides[k][2], 
                                            reg_sides[k][3], 
                                            "h"))
        elif (hv == "v" and
              reg_sides[k][0] == i+1 and 
              reg_sides[k][1] == j and
              reg_sides[k][2] == ori_x+1 and 
              reg_sides[k][3] == ori_y):
            res.append(find_connected_sides(reg_sides, 
                                            reg_sides[k][0], 
                                            reg_sides[k][1], 
                                            reg_sides[k][2], 
                                            reg_sides[k][3], 
                                            "v"))    
    return(list(set(flatten_nested_list(res))))

In [197]:
res = 0
for region in regions:
    #print(data[region[0]])
    reg_sides = find_sides(region, data)
    reg_sides = sorted(reg_sides , key=lambda k: [k[0], k[1]])

    sides = []
    all_sides = []

    for reg_side_coord in reg_sides:
        if reg_side_coord in all_sides:
            continue
        h_sides = find_connected_sides(reg_sides, 
                                       reg_side_coord[0], 
                                       reg_side_coord[1],
                                       reg_side_coord[2],
                                       reg_side_coord[3], "h")
        h_sides = flatten_nested_list(h_sides)
        v_sides = find_connected_sides(reg_sides, 
                                       reg_side_coord[0], 
                                       reg_side_coord[1],
                                       reg_side_coord[2],
                                       reg_side_coord[3], 
                                       "v")
        v_sides = flatten_nested_list(v_sides)

        if len(h_sides) == 1 and len(v_sides) == 1:
            sides.append(reg_side_coord)
            all_sides.append(reg_side_coord)
            #print("Her1", reg_side_coord)
        elif len(h_sides) > 1:
            sides.append(h_sides)
            all_sides.append(h_sides)
            #print("Her2", h_sides)
        elif len(v_sides) > 1:
            sides.append(v_sides)
            all_sides.append(v_sides)
            #print("Her3", v_sides)
        
        all_sides = flatten_nested_list(all_sides)
    res += len(region)*len(sides)
res        